In [5]:
import numpy as np
import pickle
import pandas as pd

from IPython.core.display import display

In [6]:
with open('../../raw_data/batches.meta', 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')

def shorten(v: str):
    if v == 'airplane':
        return 'plane'
    if v == 'automobile':
        return 'car'
    return v

label_names = [ shorten(v.decode("utf-8")) for v in dict[b'label_names'] ]

print(label_names)

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

batch1 = unpickle("../../raw_data/data_batch_1")
batch2 = unpickle("../../raw_data/data_batch_2")
batch3 = unpickle("../../raw_data/data_batch_3")
batch4 = unpickle("../../raw_data/data_batch_4")
batch5 = unpickle("../../raw_data/data_batch_5")
test_batch = unpickle("../../raw_data/test_batch")

def load_data0(btch):
    labels = btch[b'labels']
    imgs = btch[b'data'].reshape((-1, 32, 32, 3))

    res = []
    for ii in range(imgs.shape[0]):
        img = imgs[ii].copy()
        #img = np.transpose(img.flatten().reshape(3,32,32))
        img = np.fliplr(np.rot90(np.transpose(img.flatten().reshape(3,32,32)), k=-1))
        res.append(img)
    imgs = np.stack(res)
    return labels, imgs

def load_data():
    X_train_l = []
    y_train_l = []
    for ibatch in [batch1, batch2, batch3, batch4, batch5]:
        labels, imgs = load_data0(ibatch)
        X_train_l.append(imgs)
        y_train_l.extend(labels)
    X_train = np.vstack(X_train_l)
    y_train = np.vstack(y_train_l)

    X_test_l = []
    y_test_l = []
    labels, imgs = load_data0(test_batch)
    X_test_l.append(imgs)
    y_test_l.extend(labels)
    X_test = np.vstack(X_test_l)
    y_test = np.vstack(y_test_l)


    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_data()

lookup_names = np.vectorize(lambda i: label_names[i])

y_train = lookup_names(y_train)
y_test = lookup_names(y_test)

def wrap(X, y):
    return {
        'images': X,
        'labels': y.reshape(-1)
    }

with open('../../data/train.pkl', 'wb') as fo:
    pickle.dump(wrap(X_train, y_train), fo)

with open('../../data/test_with_labels.pkl', 'wb') as fo:
    pickle.dump(wrap(X_test, y_test), fo)

with open('../../data/test.pkl', 'wb') as fo:
    pickle.dump(X_test, fo)

y_test_df = pd.DataFrame(y_test, columns=['label'])
y_test_df.to_csv('../../data/test_labels.csv', header=True, index_label='id')


['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
